## Matcha Litteraturbanken personer utan bild i WD med SPA
Nedan test att hitta kandidater som saknar bild och är kopplade till Litteratur banken me saknar bild

* denna [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/Litteraturbanken%20SPA.ipynb)
* [video](https://youtu.be/j8Pe-7E4pT0)
  * se även SPA2Common / [video](https://www.youtube.com/watch?v=B3moN36PsNo)
  * [Notebook samma som Litteraturbanken LIBRISXL](https://github.com/salgo60/spa2Commons/blob/main/Notebook/Litteraturbanken%20-%20samma%20som.ipynb)

In [5]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)


Last run:  2022-12-31 10:42:38.986528


In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import pandas as pd
import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

#https://w.wiki/4J4B
queryLittNoPic = """SELECT ?item ?itemLabel (year(?born) as ?year) ?statements ?Litteraturbankenid
 (CONCAT(?itemLabel," ",str(year(?born))) AS ?search) WHERE {
   ?item wdt:P5101 ?Litteraturbankenid ;
         wikibase:statements ?statements .
 # FILTER (?statements <100)
  minus {?item wdt:P18 ?img}
  OPTIONAL{?item wdt:P569 ?born}
  FILTER (year(?born) < 1900)
  FILTER (year(?born) > 1800)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}  order by desc(?statements)
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, queryLittNoPic)


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittNoPic = get_sparql_dataframe(endpoint_url, queryLittNoPic)
WDLittNoPic.shape

(404, 6)

In [3]:
WDLittNoPic

,item,itemLabel,year,statements,Litteraturbankenid,search
0,http://www.wikidata.org/entity/Q20652547,Franz Hoffmann,1814,55,HoffmannF,Franz Hoffmann 1814
1,http://www.wikidata.org/entity/Q1744240,Laura Kieler,1849,48,KielerL,Laura Kieler 1849
2,http://www.wikidata.org/entity/Q3121041,Gustaf Nordenskiöld,1868,47,NordenskiöldG,Gustaf Nordenskiöld 1868
3,http://www.wikidata.org/entity/Q389396,Johan Andersson,1820,46,AnderssonJohan,Johan Andersson 1820
4,http://www.wikidata.org/entity/Q5569210,Sixten Belfrage,1883,44,BelfrageS,Sixten Belfrage 1883
...,...,...,...,...,...,...
399,http://www.wikidata.org/entity/Q100605054,Heinrich Ortenburg,1850,5,OrtenburgH,Heinrich Ortenburg 1850
400,http://www.wikidata.org/entity/Q52577083,Gunnar Wetterberg,1863,5,WetterbergG,Gunnar Wetterberg 1863
401,http://www.wikidata.org/entity/Q52690406,J.E. Ahlstrand,1814,5,AhlstrandJE,J.E. Ahlstrand 1814
402,http://www.wikidata.org/entity/Q114115033,Josef Jonsson,1850,5,JonssonJ,Josef Jonsson 1850


In [4]:
import urllib3, json
http = urllib3.PoolManager() 
SPAdetail = "https://portrattarkiv.se/details/"
url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

for index, row  in WDLittNoPic.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "5",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
        #print(h)
        source = h["_source"]
        try:
            urlPicture = urlbasePic + id
#            urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            if score > 30:
                print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")


 Franz Hoffmann 1814 http://www.wikidata.org/entity/Q20652547

 Laura Kieler 1849 http://www.wikidata.org/entity/Q1744240

 Gustaf Nordenskiöld 1868 http://www.wikidata.org/entity/Q3121041

 Johan Andersson 1820 http://www.wikidata.org/entity/Q389396

 Sixten Belfrage 1883 http://www.wikidata.org/entity/Q5569210

 Gottfrid Carlsson 1887 http://www.wikidata.org/entity/Q5603107

 Karl Fredrik Eneberg 1841 http://www.wikidata.org/entity/Q3431539

 Elisabeth Kuylenstierna-Wenster 1869 http://www.wikidata.org/entity/Q4960199
		 48.13007    Kuylenstierna-Wenster  -  1869 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZrFA 	
		 40.784653 Elisabet   Kuylenstierna-Wenster  -  1869 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABOdow 	

 August Zacharias Collin 1833 http://www.wikidata.org/entity/Q5610869
		 34.429535 August   Collin  -  1833 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2-3g 	

 Laura Valentine 1814 http://www.wikidata.org/entity/Q6499526

 Jan de Liefde 1814 http:


 Eva Ljungberg 1850 http://www.wikidata.org/entity/Q5408529

 Johan Joachim Flodin 1814 http://www.wikidata.org/entity/Q24879251

 Richard Melander 1857 http://www.wikidata.org/entity/Q26399400
		 32.601704 Johan   Melander  -  1857 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLzhQ 	
		 32.325977 Johan   Melander  -  1857 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLzpw 	
		 32.256645 Richard   Melander  -  1857 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAAAcoQ 	
		 32.164616 Johan   Melander  -  1857 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLzpg 	
		 32.019245 Richard   Melander  -  1857 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAAAcpA 	

 Harald Liedbeck 1851 http://www.wikidata.org/entity/Q106816137

 Alfred Lidholm 1839 http://www.wikidata.org/entity/Q108455938

 Ingeborg Wallenius 1832 http://www.wikidata.org/entity/Q54819982

 Hilda Fredrika Keyser 1827 http://www.wikidata.org/entity/Q84158518
		 35.953766 Hilda   Keyser-Almstedt  -  1827 http

		 32.045845 Harald   Elovson  -  1897 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAABxNw 	

 Julius Humble 1835 http://www.wikidata.org/entity/Q16649444
		 34.528522 Julius   Humble  -  1835 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAB5bg 	
		 34.445473 Julius   Humble  -  1835 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABjYA 	
		 33.333897 Julius   Humble  -  1835 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAB5bQ 	
		 30.913223 Julius   Humble  -  1835 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAB5hA 	

 Hans Schulze 1823 http://www.wikidata.org/entity/Q16650314

 Georg Smirnoff 1840 http://www.wikidata.org/entity/Q17383968

 Oscar Busch 1844 http://www.wikidata.org/entity/Q18237961
		 34.302853 Oscar   Busch  -  1844 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAg0Q 	
		 33.484406 Oscar   Busch  -  1844 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAg0g 	
		 32.955795 Oscar   Busch  -  1844 https://portrattarkiv.se/details/seuNpYu6DKAAAAAA


 Jakob Svartengren 1809 http://www.wikidata.org/entity/Q98555939

 Betty Chytræus 1832 http://www.wikidata.org/entity/Q110761155

 August Cederqvist 1851 http://www.wikidata.org/entity/Q110761229

 Johan Adolf Ziwertz 1837 http://www.wikidata.org/entity/Q113581945
		 34.569748 Adolf   Ziwetz  -  1837 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUi1A 	

 Artur Ståhle 1881 http://www.wikidata.org/entity/Q113992457

 Jakob Fredrik Biberg 1813 http://www.wikidata.org/entity/Q5575422

 Johan Henrik Chronwall 1851 http://www.wikidata.org/entity/Q15740252

 Isak Georg Stenman 1816 http://www.wikidata.org/entity/Q6189251

 Daniel Carl Alexander Skoglund 1815 http://www.wikidata.org/entity/Q51880051

 Eva Agatha Stanislas Barnekow 1813 http://www.wikidata.org/entity/Q52554783

 Johan Daniel Högberg 1823 http://www.wikidata.org/entity/Q21515942

 Frithiof Strömmerstedt 1863 http://www.wikidata.org/entity/Q107490955

 Ottilia Westermark 1835 http://www.wikidata.org/entity/Q108374435

 Adol

		 42.816654 Mortimer   Hærén  -  1835 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAACClQ 	
		 41.784363 Mortimer   Hærén  -  1835 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAACClA 	

 Albert Wilhelm Möller 1809 http://www.wikidata.org/entity/Q109678164

 Carl Ludwig Panknin 1854 http://www.wikidata.org/entity/Q109925805

 Oscar M. Bjers 1899 http://www.wikidata.org/entity/Q100151939

 Charles Henri Backman 1801 http://www.wikidata.org/entity/Q111690143

 Fredrika Lindqvist 1846 http://www.wikidata.org/entity/Q112617726

 Gustaf Karlberg 1869 http://www.wikidata.org/entity/Q112873734

 Casper Henning Lilliecrona 1861 http://www.wikidata.org/entity/Q113583656

 Anders Olof Berglund 1857 http://www.wikidata.org/entity/Q113588517

 Robert Wilhelm Urban Virgin 1823 http://www.wikidata.org/entity/Q113592154

 Herman Nyberg 1827 http://www.wikidata.org/entity/Q113622232

 Johan Georg Lönegren 1810 http://www.wikidata.org/entity/Q113992454

 Mathilda Ahlbom 1834 http://www.wikidata